<a href="https://colab.research.google.com/github/monakeshvari/used_cars_prediction_model/blob/main/used_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
from sklearn.preprocessing import StandardScaler


# Read Data

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read data from CSV file
df = pd.read_csv('/content/drive/MyDrive/used_cars.csv')

# Exploration

In [ ]:
print("Dataset shape:", df.shape)
print("First 10 rows:\n", df.head(10))
print("Last 10 rows:\n", df.tail(10))
print("Random sample of 50 rows:\n", df.sample(50))
print("Columns in the dataset:", df.columns)
print("Data types of columns:\n", df.dtypes)
print("Basic statistics:\n", df.describe())
print("Number of unique values per column:\n", df.nunique())

Dataset shape: (8128, 13)
First 10 rows:
                                    name  year  selling_price  km_driven  \
0                Maruti Swift Dzire VDI  2014         450000     145500   
1          Skoda Rapid 1.5 TDI Ambition  2014         370000     120000   
2              Honda City 2017-2020 EXi  2006         158000     140000   
3             Hyundai i20 Sportz Diesel  2010         225000     127000   
4                Maruti Swift VXI BSIII  2007         130000     120000   
5         Hyundai Xcent 1.2 VTVT E Plus  2017         440000      45000   
6          Maruti Wagon R LXI DUO BSIII  2007          96000     175000   
7                    Maruti 800 DX BSII  2001          45000       5000   
8                      Toyota Etios VXD  2011         350000      90000   
9  Ford Figo Diesel Celebration Edition  2013         200000     169000   

     fuel seller_type transmission         owner     mileage   engine  \
0  Diesel  Individual       Manual   First Owner   23.4 kmp

# Preprocess

In [ ]:
# Drop duplicates
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.shape

(6926, 13)

In [ ]:
# Drop Nulls
df.isnull().sum()
df = df.dropna()
df.shape

(6717, 13)

## Cleaning columns

### Clean 'name' column

In [ ]:
def extract_brand(name):
    """
    Extracts the brand name from the car name.

    Args:
        name (str): The full name of the car.

    Returns:
        str: The extracted brand name.
    """
    match = r'\S+'
    return re.search(match, name).group(0)

df.insert(0, 'brand', df['name'].apply(extract_brand))
df.drop('name', axis=1, inplace=True)
print(df.head())

     brand  year  selling_price  km_driven    fuel seller_type transmission  \
0   Maruti  2014         450000     145500  Diesel  Individual       Manual   
1    Skoda  2014         370000     120000  Diesel  Individual       Manual   
2    Honda  2006         158000     140000  Petrol  Individual       Manual   
3  Hyundai  2010         225000     127000  Diesel  Individual       Manual   
4   Maruti  2007         130000     120000  Petrol  Individual       Manual   

          owner     mileage   engine   max_power                    torque  \
0   First Owner   23.4 kmpl  1248 CC      74 bhp            190Nm@ 2000rpm   
1  Second Owner  21.14 kmpl  1498 CC  103.52 bhp       250Nm@ 1500-2500rpm   
2   Third Owner   17.7 kmpl  1497 CC      78 bhp     12.7@ 2,700(kgm@ rpm)   
3   First Owner   23.0 kmpl  1396 CC      90 bhp  22.4 kgm at 1750-2750rpm   
4   First Owner   16.1 kmpl  1298 CC    88.2 bhp     11.5@ 4,500(kgm@ rpm)   

   seats  
0    5.0  
1    5.0  
2    5.0  
3    5.0  
4

<ipython-input-98-440a64fc3fb8>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Clean 'torque' column

In [ ]:
df['torque'] = df['torque'].astype('str')
df['torque'] = (df['torque']
                             .str.replace(' ', '')
                             .str.replace(',', '')
                             .str.replace('+/-500', '')
                             .str.replace('at', '@')
                             .str.replace('/', '@')
                             .str.lower())

<ipython-input-99-70087edf29eb>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-99-70087edf29eb>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
def convert_torque(torque):
    """
    Converts torque values to a uniform format.

    Args:
        torque (str): The torque value as a string.

    Returns:
        str: The cleaned torque value.
    """
    match = re.search(r'(\d+\.*\d*)', torque)
    if 'kgm' in torque:
        return f"{float(match.group(0)) * 9.80665:.2f}"  # Convert kgm to nm
    return match.group(0)

df['torque'] = df['torque'].apply(convert_torque)
print(df.head())

     brand  year  selling_price  km_driven    fuel seller_type transmission  \
0   Maruti  2014         450000     145500  Diesel  Individual       Manual   
1    Skoda  2014         370000     120000  Diesel  Individual       Manual   
2    Honda  2006         158000     140000  Petrol  Individual       Manual   
3  Hyundai  2010         225000     127000  Diesel  Individual       Manual   
4   Maruti  2007         130000     120000  Petrol  Individual       Manual   

          owner     mileage   engine   max_power  torque  seats  
0   First Owner   23.4 kmpl  1248 CC      74 bhp     190    5.0  
1  Second Owner  21.14 kmpl  1498 CC  103.52 bhp     250    5.0  
2   Third Owner   17.7 kmpl  1497 CC      78 bhp  124.54    5.0  
3   First Owner   23.0 kmpl  1396 CC      90 bhp  219.67    5.0  
4   First Owner   16.1 kmpl  1298 CC    88.2 bhp  112.78    5.0  


<ipython-input-100-94e96fef121d>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Clean 'mileage' column

In [ ]:
[print(mileage) for mileage in df['mileage']]

Streaming output truncated to the last 5000 lines.
17.8 kmpl
28.4 kmpl
20.77 kmpl
24.8 kmpl
25.83 kmpl
17.8 kmpl
20.0 kmpl
14.0 kmpl
18.49 kmpl
25.44 kmpl
14.28 kmpl
16.36 kmpl
22.0 kmpl
12.8 kmpl
18.9 kmpl
16.8 kmpl
18.9 kmpl
21.63 kmpl
15.73 kmpl
26.8 kmpl
19.62 kmpl
11.36 kmpl
20.3 kmpl
17.0 kmpl
21.63 kmpl
17.01 kmpl
24.3 kmpl
25.83 kmpl
16.55 kmpl
21.5 kmpl
13.96 kmpl
15.8 kmpl
17.3 kmpl
20.4 kmpl
22.69 kmpl
19.3 kmpl
24.3 kmpl
16.2 kmpl
12.05 kmpl
19.2 kmpl
26.0 kmpl
16.0 kmpl
23.95 kmpl
23.01 kmpl
15.37 kmpl
18.49 kmpl
20.0 kmpl
12.8 kmpl
20.77 kmpl
15.11 kmpl
16.1 kmpl
19.09 kmpl
15.96 kmpl
21.43 kmpl
18.2 kmpl
22.77 kmpl
12.05 kmpl
14.0 kmpl
17.19 kmpl
16.95 kmpl
20.89 kmpl
17.5 kmpl
22.74 kmpl
13.1 kmpl
16.02 kmpl
12.8 kmpl
13.93 kmpl
19.0 kmpl
18.16 kmpl
20.36 kmpl
22.54 kmpl
24.2 kmpl
25.0 kmpl
21.63 kmpl
22.74 kmpl
27.62 kmpl
14.0 kmpl
25.83 kmpl
17.1 kmpl
23.08 kmpl
26.21 kmpl
18.6 kmpl
18.9 kmpl
13.1 kmpl
23.4 kmpl
14.0 kmpl
15.1 kmpl
13.58 kmpl
17.3 kmpl
23.4 kmpl
15.96

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
def convert_kmkg_to_kmpl(km_per_kg, energy_content_cng, energy_content_petrol):
    """
    Convert fuel efficiency from km/kg (CNG) to kmpl (petrol).

    Args:
        km_per_kg (float): Fuel efficiency in km per kg.
        energy_content_cng (float): Energy content of CNG in MJ/kg.
        energy_content_petrol (float): Energy content of petrol in MJ/l.

    Returns:
        float: Fuel efficiency in kmpl.
    """
    return km_per_kg * (energy_content_cng / energy_content_petrol)

In [ ]:
energy_content_cng = 53.6  # MJ per kilogram
energy_content_petrol = 34.2  # MJ per liter

def convert_mileage(mileage):
    """
    Converts mileage values to kmpl.

    Args:
        mileage (str): The mileage value as a string.

    Returns:
        float: The cleaned mileage value in kmpl.
    """
    match = re.search(r'(\d+\.*\d+)', mileage)
    if 'kmpl' not in mileage:
        return float(f"{convert_kmkg_to_kmpl(float(match.group(0)), energy_content_cng, energy_content_petrol):.2f}")
    return float(match.group(0))

df['mileage'] = df['mileage'].apply(convert_mileage)
print(df.head())

     brand  year  selling_price  km_driven    fuel seller_type transmission  \
0   Maruti  2014         450000     145500  Diesel  Individual       Manual   
1    Skoda  2014         370000     120000  Diesel  Individual       Manual   
2    Honda  2006         158000     140000  Petrol  Individual       Manual   
3  Hyundai  2010         225000     127000  Diesel  Individual       Manual   
4   Maruti  2007         130000     120000  Petrol  Individual       Manual   

          owner  mileage   engine   max_power  torque  seats  
0   First Owner    23.40  1248 CC      74 bhp     190    5.0  
1  Second Owner    21.14  1498 CC  103.52 bhp     250    5.0  
2   Third Owner    17.70  1497 CC      78 bhp  124.54    5.0  
3   First Owner    23.00  1396 CC      90 bhp  219.67    5.0  
4   First Owner    16.10  1298 CC    88.2 bhp  112.78    5.0  


### Clean 'engine' & 'max_power' column

In [ ]:
def match_return(column):
    """
    Extracts numeric value from a string column.

    Args:
        column (str): The column value as a string.

    Returns:
        str: The extracted numeric value.
    """
    match = re.search(r'(\d+\.*\d*)', column)
    return match.group(0)

In [ ]:
for engine in df['engine']:
  if not 'CC' in engine:
    print(engine)

In [ ]:
# Clean 'engine' column
df['engine'] = df['engine'].apply(match_return)
df.head()

,brand,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248,74 bhp,190,5.0
1,Skoda,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498,103.52 bhp,250,5.0
2,Honda,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497,78 bhp,124.54,5.0
3,Hyundai,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396,90 bhp,219.67,5.0
4,Maruti,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298,88.2 bhp,112.78,5.0


In [ ]:
for max_power in df['max_power']:
  if not 'bhp' in max_power:
    print(max_power)

In [ ]:
# Clean 'max_power' column
df['max_power'] = df['max_power'].apply(match_return)
df.head()

,brand,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248,74,190,5.0
1,Skoda,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498,103.52,250,5.0
2,Honda,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497,78,124.54,5.0
3,Hyundai,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396,90,219.67,5.0
4,Maruti,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298,88.2,112.78,5.0


## Change column data types

In [ ]:
df['engine'] = df['engine'].astype('int')
df['max_power'] = df['max_power'].astype('float')
df['torque'] = df['torque'].astype('float')
df['seats'] = df['seats'].astype('int')

## Outlier data

In [ ]:
def check_outlier_column_by_plotly(data, columns):
  fig = px.box(data, y = columns)
  fig.show()

check_outlier_column_by_plotly(df, ['year', 'selling_price', 'km_driven', 'mileage', 'engine', 'max_power', 'torque', 'seats'])

In [ ]:
def remove_outliers(data):
  Q1 = data.quantile(0.25)
  Q3 = data.quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  return data[(data >= lower_bound) & (data <= upper_bound)]

columns = ['selling_price', 'km_driven']
for col in columns:
  df[col] = remove_outliers(df[col])

df.isnull().sum()    # Replace outliers with Nans


brand              0
year               0
selling_price    306
km_driven        161
fuel               0
seller_type        0
transmission       0
owner              0
mileage            0
engine             0
max_power          0
torque             0
seats              0
dtype: int64

In [ ]:
def fillna(data):
  data.fillna(value = {
      'selling_price' : data.selling_price.mean(),
      'km_driven' : data.km_driven.mean()
  }, inplace = True)
  return data

df = fillna(df)

## Encoder

In [ ]:
# One-Hot Encoding

def one_hot_encoder(data, columns):
  return pd.get_dummies(data, columns = columns)

df = one_hot_encoder(df, ['transmission'])

df['transmission_Automatic'] = df['transmission_Automatic'].astype('int')
df['transmission_Manual'] = df['transmission_Manual'].astype('int')

In [ ]:
# Label Encoding

def label_encoder(data, columns):
  le = LabelEncoder()
  for col in columns:
    data[col] = le.fit_transform(data[col])
  return data

df = label_encoder(df, ['brand', 'year', 'fuel', 'seller_type', 'owner', 'seats'])
df

,brand,year,selling_price,km_driven,fuel,seller_type,owner,mileage,engine,max_power,torque,seats,transmission_Automatic,transmission_Manual
0,20,20,450000.0,145500.0,1,1,0,23.40,1248,74.00,190.00,2,0,1
1,26,20,370000.0,120000.0,1,1,2,21.14,1498,103.52,250.00,2,0,1
2,10,12,158000.0,140000.0,3,1,4,17.70,1497,78.00,124.54,2,0,1
3,11,16,225000.0,127000.0,1,1,0,23.00,1396,90.00,219.67,2,0,1
4,20,13,130000.0,120000.0,3,1,0,16.10,1298,88.20,112.78,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8121,20,19,260000.0,50000.0,3,1,2,18.90,998,67.10,90.00,2,0,1
8122,11,20,475000.0,80000.0,1,1,2,22.54,1396,88.73,219.70,2,0,1
8123,11,19,320000.0,110000.0,3,1,0,18.50,1197,82.85,113.70,2,0,1
8124,11,13,135000.0,119000.0,1,1,1,16.80,1493,110.00,235.36,2,0,1


## Define X and y


In [ ]:
X = df.drop('selling_price', axis=1)
y = df['selling_price']

## Standardization

In [ ]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the data
scaled_X = scaler.fit_transform(X)

# Convert the scaled features back to a DataFrame, preserving the column names
scaled_X_df = pd.DataFrame(scaled_X, columns=X.columns)
scaled_X_df

# Combine the scaled features and the target into one DataFrame
result_df = pd.concat([scaled_X_df, y.reset_index(drop=True)], axis=1)
result_df